In [149]:
from functools import reduce
from time import time

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances

#from sklearn.metrics import jaccard_score # scikit 0.21
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import accuracy_score

In [5]:
# X shoudl be a numpy matrix, very likely sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
# T1 > T2 for overlapping clusters
# T1 = Distance to centroid point to not include in other clusters
# T2 = Distance to centroid point to include in cluster
# T1 > T2 for overlapping clusters
# T1 < T2 will have points which reside in no clusters
# T1 == T2 will cause all points to reside in mutually exclusive clusters
# Distance metric can be any from here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
# filemap may be a list of point names in their order in X. If included, row numbers from X will be replaced with names from filemap. 
 
def canopy(X, T1, T2, distance_metric='euclidean', filemap=None):
    canopies = dict()
    X1_dist = pairwise_distances(X, metric=distance_metric)
    canopy_points = set(range(X.shape[0]))
    while canopy_points:
        point = canopy_points.pop()
        i = len(canopies)
        canopies[i] = {"c":point, "points": list(np.where(X1_dist[point] < T2)[0])}
        canopy_points = canopy_points.difference(set(np.where(X1_dist[point] < T1)[0]))
    if filemap:
        for canopy_id in canopies.keys():
            canopy = canopies.pop(canopy_id)
            canopy2 = {"c":filemap[canopy['c']], "points":list()}
            for point in canopy['points']:
                canopy2["points"].append(filemap[point])
            canopies[canopy_id] = canopy2
    return canopies

In [310]:
def euclideanDistance(vector1, vector2):
        #print(vector1)
        #print(vector2)
        return np.sqrt(np.sum(np.power(vector1-vector2, 2)))

def getDistance(row_center, row_sample):
        #print(row_center)
        row_center = np.asarray(row_center)
        #row_center = np.asarray(row_sample)
        return euclideanDistance(row_center, row_sample)

def getSquaredError(data, kmeans, k):
    distances = []
    for i in range(k): # Qtd de clusters
        distance = 0
        for index_labels, value_labels in enumerate(kmeans.labels_): #kmeans.labels_ possui o cluster de cada elemento
            if (i == value_labels):
                #print(value_labels)
                distance = distance + getDistance(kmeans.cluster_centers_[value_labels], data[index_labels])
        distances.append(distance) #Erro quadratico medio de cada cluster
    distances = np.asarray(distances)
    error = np.sum(distances)
    return error

In [274]:
# Definition 1
def get_mean_distance(distance_matrix):
    n = len(distance_matrix)
    return distance_matrix.sum() / (n* (n-1)) if n > 1 else distance_matrix.sum()

In [266]:
# Definition 2
def get_density(distance_matrix, mean_distance):
    densities = np.zeros(len(distance_matrix))
    
    matrix = distance_matrix - mean_distance
    matrix[matrix >= 0] = 0
    matrix[matrix < 0] = 1

    for i, row_i in enumerate(matrix):
        densities[i] = row_i.sum()
    return densities

In [267]:
#Definition 3
def cluster_tightness(distance_matrix, mean_distance):
    tightness = np.zeros(len(distance_matrix))
    
    for i, row_i in enumerate(distance_matrix):
        mean = row_i[row_i < mean_distance].mean()
        tightness[i] = 0 if mean == 0 else 1 / mean
    return tightness

In [268]:
#Definition 4
def clusters_dissimilarity(densities, distance_matrix):
    n = len(distance_matrix)
    dissimilarity = np.zeros(n)
    for i in range(n):
        d_row = distance_matrix[i]
        d_row = np.delete(d_row, np.where(d_row == 0))
        dissimilarity[i] = d_row.min()
    max_id = np.where(densities == densities.max())
    dissimilarity[max_id] = distance_matrix[max_id].max()
    return dissimilarity

In [269]:
def remove_cluster(c, data, distance_matrix, mean_distance):
    center_row = distance_matrix[c]
    center_row[center_row < mean_distance] = 0
    
    #pontos a serem removidos
    points = np.where(center_row == 0)
    data = np.delete(data, points, axis=0)
    
    distance_matrix = np.delete(distance_matrix, points, axis=0)
    distance_matrix = np.delete(distance_matrix, points, axis=1)
    
    return data, distance_matrix

In [283]:
def removeOutliers(aux_D, densities, inv_a, s, meanDis):
    #remove elemento com densidade = 1 e que o s[i] seja maior que o raio
    outliers = []
    for i, row_i in enumerate(aux_D.values):
        if densities[i] == 1 and s[i] == max(s) and aux_D.shape[0] > 1:
            outliers.append(i)
    aux_D.drop(outliers, inplace=True) #removendo outliers
    aux_D.reset_index(drop=True, inplace=True)
    densities = np.delete(densities, outliers, 0)
    inv_a = np.delete(inv_a, outliers, 0)
    s = np.delete(s, outliers, 0)
    return aux_D, densities, inv_a, s

In [319]:
def densityCanopy(data, distance_matrix):
    centers = []
    db = data.copy()
    d_matrix = distance_matrix.copy()
    
    #Step 1
    mean_distance = get_mean_distance(d_matrix)
    density_set = get_density(d_matrix, mean_distance)
    c = np.argmax(density_set)
    centers.append(db[c])
    db, d_matrix = remove_cluster(c, db, d_matrix, mean_distance)
    
    #Step 2
    mean_distance = get_mean_distance(d_matrix)
    density_set = get_density(d_matrix, mean_distance)
    tightness_set = cluster_tightness(d_matrix, mean_distance)
    dissimilarity_set = clusters_dissimilarity(density_set, d_matrix)
    w_set = density_set * tightness_set * dissimilarity_set # Product Weight
    c = np.argmax(w_set)
    centers.append(db[c])
    db, d_matrix = remove_cluster(c, db, d_matrix, mean_distance)
    
    #Step 3
    while len(db) > 1:
        
        mean_distance = get_mean_distance(d_matrix)
        density_set = get_density(d_matrix, mean_distance)
        tightness_set = cluster_tightness(d_matrix, mean_distance)
        
        # Distancia das instancias para os centros já escolhidos
        clusters_distance = pairwise_distances(db, centers, metric='euclidean')
        
        w_set = density_set * tightness_set
        for i in range(len(db)):
            w_set[i] = reduce(lambda x, y: x * y, clusters_distance[i] * w_set[i])
        
        c = np.argmax(w_set)
        centers.append(db[c])
        db, d_matrix = remove_cluster(c, db, d_matrix, mean_distance)
        
    if len(db) == 1:
        centers.append(db[0])
    
    return centers

In [328]:
def run():
    # demo_data = np.random.random_sample((2000, 5))
    # print(demo_data.shape)
#     data = auxData
#     demo_data = data.values
    
    files = ["soybean-small", "iris", "wine", "segmentation", "ionosphere"]
    ks = [4,3,3,7,2]
    kmeansTypes = ["random", "k-means++", "densityCanopy"]
    
    for kmeansType in kmeansTypes:
        print ("--------- "+ kmeansType +" test ---------")
        
        for index, file in enumerate(files):
            clustering_time = []
            clustering_error = []
            print ("\n----- "+file+" -----\n")
            data = pd.read_csv("datasets/"+file+".data", header=None)
            if file == "segmentation" or file == "wine": #Target eh na primeira coluna
                target = data.iloc[:,0]
                data = data.iloc[:,1:]
            else: #Target na última coluna
                target = data.iloc[:,-1]
                data = data.iloc[:,:-1]
            data = data.values
            distance_matrix = pairwise_distances(data, metric='euclidean')
            k = ks[index]

            start = time()
            if kmeansType == "densityCanopy":
                centers = np.asarray(densityCanopy(data, distance_matrix))
                l = len(centers)
                kmeans = KMeans(n_clusters=l, random_state=100, init=centers, n_init=1, max_iter=100).fit(data)
            else:
                kmeans = KMeans(n_clusters=k, random_state=100, init=kmeansType, n_init=10, max_iter=100).fit(data)

            error = getSquaredError(data, kmeans, k)
            clustering_error.append(error)
            #print("Erro:", error)

            end = time()
            clustering_time.append(end - start)

            print ("\n---Tempos---")
            print ("" + kmeansType, clustering_time[0])

            print ("\n---Erros---")
            print ("" + kmeansType, clustering_error[0])

In [329]:
run()

--------- random test ---------

----- soybean-small -----


---Tempos---
random 0.013199329376220703

---Erros---
random 96.99203638192614

----- iris -----


---Tempos---
random 0.024935007095336914

---Erros---
random 97.32592423430009

----- wine -----


---Tempos---
random 0.031957149505615234

---Erros---
random 16555.679416032723

----- segmentation -----


---Tempos---
random 0.1558551788330078

---Erros---
random 150225.55247187975

----- ionosphere -----


---Tempos---
random 0.01795196533203125

---Erros---
random 796.4666759281233
--------- k-means++ test ---------

----- soybean-small -----


---Tempos---
k-means++ 0.015027999877929688

---Erros---
k-means++ 96.99203638192614

----- iris -----


---Tempos---
k-means++ 0.01749110221862793

---Erros---
k-means++ 97.32592423430009

----- wine -----


---Tempos---
k-means++ 0.019376039505004883

---Erros---
k-means++ 16555.679416032723

----- segmentation -----


---Tempos---
k-means++ 0.1493971347808838

---Erros---
k-means++

In [284]:
mu, sigma = 0, 0.1 
# creating a noise with the same dimension as the dataset (2,2) 
noise = np.random.normal(mu, sigma, [2,2]) 
print(noise)

[[ 0.09386941  0.07403991]
 [-0.10084067  0.18505222]]


u A X = 0, u Y A = 0
σ x = σ y = 2

u B X = 5, u Y B = 1
σ x = 1 , σ y = 2

u C X = 5, u C Y = −2
σ x = σ y = 1